In [5]:
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# tf.keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()

from tensorflow.keras import layers

import warnings

warnings.filterwarnings("ignore")

In [6]:
from dataclasses import dataclass


@dataclass
class Parameters:
    learning_rate: float = 0.001
    batch_size: int = 32
    data_dir = Path('/data')
    train_dir = data_dir / 'flower_photos'
    verbose = 1
    epochs = 20
    img_size = 224
    img_shape = (img_size, img_size, 3)
    class_names = None
    seed = 42

config = Parameters()


In [7]:
train_dir = 'data/flower_photos'

In [8]:
# Your existing code
batch_size =32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = train_ds.class_names

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [16]:
#transfer learning using denseNet121

denseNet121 = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling='avg'
)
denseNet121.trainable = False

In [21]:
#create model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(224,224,3)),
    layers.RandomFlip("horizontal_and_vertical", seed=42),
    layers.RandomRotation(0.3, seed=42),
    denseNet121,
    layers.Dense(5)
])

In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_1 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 dense_3 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 7042629 (26.87 MB)
Trainable params: 5125 (20.02 KB)
Non-trainable params: 7037504 (26.85 MB)
_________________________________________________________________


In [23]:
# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#define loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#compile model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-8, mode='min')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoints/model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
callbacks = [reduce_lr, model_checkpoint]

In [24]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [25]:
#fit model
epochs=10
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=epochs,
    callbacks=callbacks,
)

Epoch 1/10
92/92 [==============================] - ETA: 0s - loss: 2.5572 - accuracy: 0.4309
Epoch 1: val_accuracy improved from -inf to 0.57766, saving model to checkpoints/model.h5
92/92 [==============================] - 153s 2s/step - loss: 2.5572 - accuracy: 0.4309 - val_loss: 1.5724 - val_accuracy: 0.5777 - lr: 0.0010
Epoch 2/10
53/92 [================>.............] - ETA: 46s - loss: 1.5288 - accuracy: 0.5619

KeyboardInterrupt: 